# Hashing

In [ ]:
import hashlib
import os
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [3]:
hashlib.algorithms_available

{'blake2b',
 'blake2s',
 'md5',
 'md5-sha1',
 'ripemd160',
 'sha1',
 'sha224',
 'sha256',
 'sha384',
 'sha3_224',
 'sha3_256',
 'sha3_384',
 'sha3_512',
 'sha512',
 'sha512_224',
 'sha512_256',
 'shake_128',
 'shake_256',
 'sm3'}

## Speed tests

In [ ]:
def create_random_file(filename: str, size_bytes: int):
    """Create a file with random data of size_bytes for hashing tests."""
    with open(filename, 'wb') as f:
        f.write(os.urandom(size_bytes))


# Examples for different sizes:
create_random_file('../data/temp/1kb.txt', 1 * 1024)
create_random_file('../data/temp/10kb.txt', 10 * 1024)
create_random_file('../data/temp/100kb.txt', 100 * 1024)
create_random_file('../data/temp/1000kb.txt', 1024 * 1024)
create_random_file('../data/temp/10000kb.txt', 10 * 1024 * 1024)
create_random_file('../data/temp/100000kb.txt', 100 * 1024 * 1024)
create_random_file('../data/temp/1000000kb.txt', 1024 * 1024 * 1024)


In [5]:
def calculate_file_hash(
    hash_algo: str, file_path: str | Path, chunk_size: int = 256 * 1024
) -> str:
    """Calculate hash of a file using the given hash algorithm."""
    if hash_algo not in hashlib.algorithms_available:
        raise ValueError(f'Hash algorithm {hash_algo} is not available in hashlib.')
    if isinstance(file_path, str):
        file_path = Path(file_path)
    try:
        hash_algo = hashlib.new(hash_algo)
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                hash_algo.update(chunk)
        return hash_algo.hexdigest()
    except (IOError, OSError) as e:
        print(e)
        return ''


In [6]:
results = []

for algo in hashlib.algorithms_available:
    # Skip shake algorithms (they require a length parameter)
    if algo.startswith('shake'):
        continue
    for f in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000]:
        try:
            filepath = f'../data/temp/{f}kb.txt'

            # Time the hashing here
            start_time = time.time()
            hash_value = calculate_file_hash(algo, filepath, 10 * 1024 * 1024)
            end_time = time.time()

            time_taken = end_time - start_time

            results.append(
                {
                    'algorithm': algo,
                    'file_size_mb': f,
                    'time_seconds': time_taken,
                    'hash': hash_value,
                }
            )
        except Exception as e:
            print(f'Error with {algo} on {f}mb: {e}')

# Create DataFrame
df_results = pd.DataFrame(results)

In [9]:
df_results


algorithm  file_size_mb  time_seconds  \
0         md5             1      0.000856   
1         md5            10      0.000748   
2         md5           100      0.000637   
3         md5          1000      0.004551   
4         md5         10000      0.029626   
..        ...           ...           ...   
114    sha224           100      0.000242   
115    sha224          1000      0.002565   
116    sha224         10000      0.027467   
117    sha224        100000      0.250317   
118    sha224       1000000      2.335254   

                                                  hash  
0                     c272bf8e9ead02fba857ee7c63a939d5  
1                     28cbed343629a0b2c84f7ffbb161ff82  
2                     4435febee7a720b066f28c0f14de4822  
3                     6dc77ae66574cd17e0e1448f7d22b1ff  
4                     52edee4ce29a3ff50d1e035bf146a6cf  
..                                                 ...  
114  47bb039c67d664638c0b6165458e7ba4e8f96616f2efe1...  
115  a1f2487c1840c358be4cdd95b88ca62a44c6d22b1b6447...  
116  b9243f7efa2734ee36f864608316e7389728729dab907b...  
117  7f1caf67496f82f9db9770feffd5d2a8dda06aa2e5abbc...  
118  415565ba1ce61573ba11d32fb7143b7aaf0ea2c9fb3978...  

[119 rows x 4 columns]

In [10]:
avg_times = df_results.groupby('algorithm')['time_seconds'].mean().sort_values()
avg_times

algorithm
sha1          0.186484
blake2b       0.230512
md5           0.235958
sha512        0.249567
sha512_256    0.251228
sha512_224    0.251552
sha384        0.251742
blake2s       0.354930
sha256        0.362684
sha224        0.373719
md5-sha1      0.388971
ripemd160     0.422244
sha3_224      0.428085
sha3_256      0.436395
sha3_384      0.565642
sm3           0.626717
sha3_512      0.806291
Name: time_seconds, dtype: float64

In [13]:
top_algorithms = avg_times.head(8).index

plt.figure(figsize=(12, 8))
for algo in top_algorithms:
    algo_data = df_results[df_results['algorithm'] == algo]
    plt.plot(
        np.log10(algo_data['file_size_mb']),
        algo_data['time_seconds'],
        marker='o',
        linewidth=2,
        label=algo,
    )

plt.xlabel('log10(File Size (MB))', fontsize=12)
plt.ylabel('Time (seconds)', fontsize=12)
plt.title('Hash Performance: Top Algorithms', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

For file deduplication, the key requirement is speed rather than cryptographic strength. The goal is to quickly detect identical files, not to protect against deliberate hash collisions. SHA1 emerges as the fastest algorithm in our performance tests, making it an excellent choice for this purpose because it is extremely fast, widely supported, and its collision risk is negligible in non-adversarial contexts.
   
Modern alternatives like BLAKE2b offer competitive performance with stronger hash properties, making them excellent choices for large-scale or performance-critical deduplication systems where both speed and security are important considerations.   

Key findings from our benchmarks:
- SHA1 consistently outperformed other algorithms across all file sizes tested
- MD5 remains a solid second choice for maximum compatibility
- BLAKE2 algorithms provide the best balance of speed and cryptographic strength
- Traditional algorithms like SHA256, while secure, are significantly slower for large-scale deduplication tasks